# 1. Tokenisation

In deze notebook laden we de commentaren van de enquête in en passen we tokenisation toe: het splitsen van de tekst in aparte woorden/entiteiten.

## 1.1 Imports

We laden uitbreidingen in die we nodig hebben om de dataset in te laden.

In [14]:
import pandas as pd
from spacy_download import load_spacy

## 1.2 Dataset inladen

We laden de dataset in en tonen een preview.

In [2]:
df = pd.read_csv("data/data_text.csv")

In [3]:
df.head()

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,QPERS_LING_JOB,QPERS_READING,Q53,Q55,QPERS_IDEAS,QPERS_ORIGIN,QPERS_COMPLAIN,QPERS_UPDATE,QPERS_WIN,QPERS_EMAIL
0,2022-05-19 01:32:15,2022-05-19 01:49:25,IP Address,46.183.103.8,100,1030,True,2022-05-19 01:49:27,R_2wjiSHblELDyZMY,NaN,...,...wél met taal bezig,100.0,Nee,Ja,voorlezen met acteurs of auteurs themagerelate...,via auteurs,NaN,Ik wil graag op de hoogte gehouden worden over...,Ik wil kans maken op een prijs.,gaeaschoeters@yahoo.com
1,2022-05-19 02:00:47,2022-05-19 02:05:00,IP Address,92.110.70.15,100,252,True,2022-05-19 02:05:02,R_1I6pSyFmgXFtRjH,NaN,...,...wél met taal bezig,20.0,Nee,Ja,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-05-19 01:55:03,2022-05-19 02:05:54,IP Address,81.242.7.76,100,651,True,2022-05-19 02:05:56,R_2OVxUcnSQ8BFaqa,NaN,...,...wél met taal bezig,50.0,Nee,Ja,"Tijdschriftnummers, debatten, festivals",Facebook,NaN,Ik wil graag op de hoogte gehouden worden over...,NaN,michielleen@hotmail.com
3,2022-05-19 01:59:38,2022-05-19 02:31:20,IP Address,84.198.88.221,100,1901,True,2022-05-19 02:31:21,R_2b0CIkQZZJq6ey6,NaN,...,...wél met taal bezig,20.0,Nee,Ja,NaN,NaN,NaN,Ik wil graag op de hoogte gehouden worden over...,Ik wil kans maken op een prijs.,maksimmarissen@gmail.com
4,2022-05-19 03:08:29,2022-05-19 03:15:49,IP Address,131.211.188.193,100,440,True,2022-05-19 03:15:49,R_6M3bVJZ8xxgVL5T,NaN,...,...wél met taal bezig,NaN,Nee,Ja,NaN,Neerlandistiek.nl,NaN,Ik wil graag op de hoogte gehouden worden over...,Ik wil kans maken op een prijs.,i.fuchs@uu.nl


## 1.3 Tokenisation

[spaCy](https://spacy.io/) is een bekende package om tekst te parsen en verwerken. Met spaCy kan je héél veel (POS-taggen, tokenisen, lemmatisen, named entity recognition). Wij gaan gewoon alles opsplitsen in woordjes.

Ik heb [load_spacy](https://github.com/BramVanroy/spacy_download) van collega Bram Vanroy gebruikt, omdat dit automatisch ook het model downloadt als je het nog niet hebt. Verder werkt alles zoals je zou verwachten van spaCy.

In [8]:
nlp = load_spacy("nl_core_news_lg", exclude=["parser", "tagger"])

Ik toon hoe je een zin kunt tokeniseren met spaCy:

In [9]:
doc = nlp("Freek Van de Velde neemt deel aan de fantastische taalkundequiz.")
for token in doc:
    print(token.text)

Freek
Van
de
Velde
neemt
deel
aan
de
fantastische
taalkundequiz
.


Nu is het de bedoeling dat we hetzelfde doen met de commentaren uit de enquête. Ik geef een voorzet:

In [16]:
# We itereren over elke rij van de dataset
for index, row in df.iterrows():
    # Je kunt een kolom opvragen door die te indexeren
    # bv. row["Q9_ELAB"]

    # Soms is er niks ingevuld, dan is de waarde van de kolom "nan" of "null"
    # pd.isnull() kan hiervoor testen (True/False)
    print(row["Q9_ELAB"])

    # Tokenisation

zonder canon kan je niks plaatsen of je nergens tegen verzetten
nan
nan
Vanuit cultuurhistorisch opzicht is het belangrijk te weten welke werken impact hadden in verschillende periodes die aan de onze voorafgaan. Het helpt om hedendaagse identiteitsdebatten te begrijpen.
nan
We hebben de verantwoordelijkheid om het literair erfgoed op een voor leerlingen relevante en urgente manier in het onderwijs aan te bieden. 
lezen mag je best verplichten, maar niet wát er gelezen moet worden
nan
Het is belangrijk dat leerlingen worden uitgenodigd om hun leeservaringen te verrijken maar geen enkel boek is relevant voor alle leerlingen.
nan
nan
Zeker in het onderwijs is een (lichte) dwang soms noodzakelijk. 
een overzicht bieden zonder het verplichte aspect erbij, geen dwang, werkt averechts
Als je als tiener weinig leest is het volgens mij echt geen goed idee om verplicht te worden om een klassieker te lezen, want die boeken zijn moeilijker en "saaier" 
Verplichte leeslijsten leiden vaak tot afkee